In [1]:
from cuppa3_cc import cc
from exp2bytecode_interp import interp

## Testing the compiler

In [2]:
add = \
'''
declare add(a,b) 
{
    return a+b;
}

declare x = add(3,2);
put x;
'''

seqsum = \
'''
declare add(a,b) 
{
    return a+b;
}

declare seqsum(n) 
{
    
    declare i = 1;
    declare sum = 0;
        
    while (i <= n) 
    {
        sum = add(sum,i);
        i = i + 1;
    }
        
    put sum;
}

seqsum(10);
'''

and_prog = \
'''
declare and(a,b)
{
    return a*b;
}

put and(1,1);
put and(1,0);
put and(0,1);
put and(0,0);
'''

fact = \
'''
// computes the factorial of x = 3
declare x = 3;
declare y = 1;
while (1 <= x) 
{
      y = y * x;
      x = x - 1;
}
put y;
'''

factrec = \
'''
// recursive implementation of factorial
declare fact(x) 
{
     if (x <= 1)
        return 1;
     else 
        return fact(x-1) * x;
}

put fact(3);
'''

fold = \
'''
declare x = (3 + 2) / 5;
put x;
'''


func1 = \
'''
declare f () {
 put(1001);
}

f();
'''

In [3]:
interp(cc(add))

> 5


In [4]:
interp(cc(seqsum))

> 20


In [5]:
interp(cc(and_prog))

> 1
> 0
> 0
> 0


In [6]:
interp(cc(fact))

> 6


In [7]:
interp(cc(factrec))

> 6


In [ ]:
cc(fold)

In [ ]:
cc(func1)

Testing our interpreter on the recursive computation of the Fibonacci sequence

In [ ]:
fib = \
'''
declare fib(n)
{    
    if (n == 0)
        return 0;
    else if (n == 1)
        return 1;
    else
        return fib(n-1) + fib(n-2);
}

// fibonacci sequence: 0,1,1,2,3,5,8,13,21,34,55,89,144,...
declare i = 0
while (i <= 12)
{
    put fib(i)
    i = i + 1
}
'''

In [ ]:
cc(fib)

## Explaining the throwing of Return Values

In [ ]:
from cuppa3_lex import lexer
from cuppa3_cc_frontend_gram import parser
from cuppa3_cc_state import state
from cuppa3_cc_tree_rewrite import walk as rewrite
from cuppa3_cc_codegen import walk as codegen
from cuppa3_cc_output import output
from grammar_stuff import dump_AST
from cuppa3_cc import cc
from pprint import pprint

In [ ]:
state.initialize()

In [ ]:
parser.parse(add, lexer=lexer)

In [ ]:
t = rewrite(state.AST)

In [ ]:
dump_AST(t)

In [ ]:
print(output(codegen(t)))

In [ ]:
inc = \
'''
declare inc(i) 
{
     return i+1;
}
'''

prog = \
'''
declare y = inc(1);
put y;
'''

In [ ]:
parser.parse(inc,lexer=lexer)

In [ ]:
dump_AST(state.AST)

In [ ]:
parser.parse(prog,lexer=lexer)

In [ ]:
dump_AST(state.AST)

In [ ]:
print(inc + prog)

In [ ]:
factfunc = \
'''
// recursive implementation of factorial
declare fact(x) 
{
     if (x <= 1)
        return 1;
     else 
        return fact(x-1) * x;
}
'''
state.initialize()
parser.parse(factfunc,lexer=lexer)
dump_AST(state.AST)